In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
class MarketMotion:
    def __init__(self, initial_price: float, drift: float, volatility: float, timestep: float, duration: float) -> None:
        self._initial_price = initial_price
        self._current_price = self._initial_price

        self._drift = drift
        self._volatility = volatility

        self._timestep = timestep
        self._duration = duration
        self._current_time = 0

        self._prices = np.empty(int(self._duration / self._timestep))

    def step(self) -> "MarketMotion":
        self._step()
        self._current_time += 1

        return self

    def run(self, duration: float | None = None) -> "MarketMotion":
        if duration is None:
            for _ in range(int((self._duration - (self._current_time * self._timestep)) / self._timestep)):
                self.step()

        return self

    def get_time_series(self) -> np.NDArray[np.float64]:
        return self._prices

    def _step(self) -> None:
        raise NotImplementedError


In [ ]:
class Payoff:
    def __init__(self, strike_price: float) -> None:
        self._strike = strike_price
        
    def get_payoff(self, stock_price_path: list[float]) -> float:
        raise NotImplementedError

In [ ]:
def run_sim(motion, *motion_args, motion_kwargs: dict[str, Any], payoffs: dict[str, Payoff] = {}, risk_free_rate: float = 0.01, number_simulations: int = 5000, plot: bool = True, ) -> float:
    price_paths = [motion(*motion_args, **motion_kwargs).run().get_time_series()
                   for _ in range(number_simulations)]

    call_payoffs = {
        key: [] for key in list(payoffs.keys())
    }

    for price_path in price_paths:
        for key in list(payoffs.keys()):
            call_payoffs[key].append(payoffs[key].get_payoff(
                price_path) / (1 + risk_free_rate))

    if plot:
        # Plot the set of generated sample paths
        for price_path in price_paths:
            plt.plot(price_path)

        # plt.xlabel("Day")
        plt.ylabel("Price")
        plt.show()

    print(
        f"Average Value: {(avg_value := np.array(price_paths)[:, -1].mean())}")
    print(f"Max Value: {np.array(price_paths)[:, -1].max()}")
    print(f"Min Value: {np.array(price_paths)[:, -1].min()}")
    # Options are in blocks of 100
    # print(f"Effective price: {(option_price := np.average(call_payoffs)*100)}")
    prices = []
    for key in list(payoffs.keys()):
        print(f"{key}: {(_price := np.array(call_payoffs[key]).mean() * 100)}")
        prices.append(_price)

    return avg_value, prices


# EXAMPLE MOTIONS AND PAYOFFS

In [ ]:
class FranchiseMotion(MarketMotion):
    DRIFT_PER_YEAR = 0.04
    VOLITILITY_PER_YEAR = 0.03
    YEARS = 10
    MONTHS_PER_YEAR = 12

    def __init__(self) -> None:
        super().__init__(
            385000,
            FranchiseMotion.DRIFT_PER_YEAR * FranchiseMotion.YEARS,
            (FranchiseMotion.VOLITILITY_PER_YEAR /
             FranchiseMotion.MONTHS_PER_YEAR) * FranchiseMotion.YEARS,
            1 / (FranchiseMotion.YEARS * FranchiseMotion.MONTHS_PER_YEAR),
            1
        )

    def _step(self) -> None:
        dWt = np.random.normal(0, math.sqrt(self._timestep))  # Brownian motion
        dYt = (self._drift * self._timestep) + \
            (self._volatility * dWt)  # Change in price

        self._current_price *= (1 + dYt)  # Add the change to the current price
        # Append new price to series
        self._prices[self._current_time] = self._current_price

class FSurgeMotion(MarketMotion):
    DRIFT_PER_YEAR = 0.04
    VOLITILITY_PER_YEAR = 0.03
    YEARS = 10
    MONTHS_PER_YEAR = 12

    def __init__(self) -> None:
        super().__init__(
            385000,
            FSurgeMotion.DRIFT_PER_YEAR * FSurgeMotion.YEARS,
            (FSurgeMotion.VOLITILITY_PER_YEAR /
             FSurgeMotion.MONTHS_PER_YEAR) * FSurgeMotion.YEARS,
            1 / (FSurgeMotion.YEARS * FSurgeMotion.MONTHS_PER_YEAR),
            1
        )

    def _step(self) -> None:
        if self._current_time >= 60:
            self._volatility = .19

        dWt = np.random.normal(0, math.sqrt(self._timestep))  # Brownian motion
        dYt = (self._drift * self._timestep) + \
            (self._volatility * dWt)  # Change in price

        self._current_price *= (1 + dYt)  # Add the change to the current price
        # Append new price to series
        self._prices[self._current_time] = self._current_price

class European_Call_Payoff(Payoff):
    def __init__(self, strike_price: float) -> None:
        super().__init__(strike_price)

    def get_payoff(self, stock_price_path: list[float]) -> float:
        stock_price = stock_price_path[-1]

        if stock_price > self._strike:
            return stock_price - self._strike
        else:
            return 0


class American_Call_Payoff(Payoff):
    def __init__(self, strike_price: float) -> None:
        super().__init__(strike_price)

    def get_payoff(self, stock_price_path: list[float]) -> float:
        stock_price_index = np.argmax(stock_price_path >= self._strike)

        if stock_price_index == 0:
            return 0
        
        stock_price = stock_price_path[stock_price_index]

        if stock_price > self._strike:
            return stock_price - self._strike
        else:
            return 0

# ACTUAL MOTIONS AND PAYOFFS
College motion, one per college
european payoff for determining value

In [ ]:
#initial price is determined by heuristic function

class CollegeEducationMotion(MarketMotion):
    #can be played around with
    DRIFT = 0
    VOLATILITY = 0
    

    def __init__(self, initial_price: float) -> None:
        super().__init__(
            initial_price, 
            CollegeEducationMotion.DRIFT, 
            CollegeEducationMotion.VOLATILITY, 
            1, #month
            4 * 12 #12 months for 4 years
        )

    def _step(self) -> None:
        dWt = np.random.normal(0, math.sqrt(self._timestep))  # Brownian motion, change this to change the motion


        dYt = (self._drift * self._timestep) + \
            (self._volatility * dWt)  # Change in price

        self._current_price *= (1 + dYt)  # Add the change to the current price
        # Append new price to series
        self._prices[self._current_time] = self._current_price

In [ ]:
def heuristic_function(college_name: str) -> tuple[float,float]:
    raise NotImplementedError("Implement me")

# Sim

In [ ]:
COLLEGES: list[str] = []

for college in COLLEGES:
    intial_education_value, desired_education_value = heuristic_function(college)
    run_sim(
        CollegeEducationMotion,
        motion_kwargs={
            "initial_price" : intial_education_value 
        },
        payoffs={
            "Education Payoff": European_Call_Payoff(strike_price=desired_education_value)
        }
    )